<a href="https://colab.research.google.com/github/ChantelleAA/Reinforcement_Learning_Oware/blob/main/classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classes being considered are Board, Player, Rules and Game State

### **Board**

In [ ]:
import numpy as np
import gym
import random

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class Board():
    def __init__(self):
        self.nrows = 2
        self.ncols = 6
        self.total_stores = 2
        self.board = 4 * np.ones((self.nrows, self.ncols))

        self.total_territories = self.nrows * self.ncols
        self.stores = np.zeros((self.total_stores,))
        self.territory_count = np.array((self.ncols, self.ncols))
        self.board_indices = list(np.ndindex(self.nrows, self.ncols))

        self.n_players = 2
        self.current_player = 1
        self.other_player = 2

    @property
    def total_seeds(self):
        return np.sum(self.board, axis=None)

    @property
    def board_format(self):
        return self.board_indices[5::-1] + self.board_indices[6:]

    def zero_rows(self):
        rows = np.array([False, False])
        for i in range(self.nrows):
            if np.sum(self.board[i, :]) == 0:
                rows[i] = True
        return rows

    def reset_board(self):
        self.board = 4 * np.ones((self.nrows, self.ncols))

    def action2pit(self, action):
        return self.board_format[action]

    def get_seeds(self, action):
        pit_index = self.action2pit(action=action)
        return self.board[pit_index]

    def set_seeds(self, action, new_value):
        pit_index = self.action2pit(action=action)
        self.board[pit_index] = new_value

    def update_seeds(self, action, new_value):
        pit_index = self.action2pit(action=action)
        self.board[pit_index] += new_value

    def distribute_seeds(self, action):
        # should just change the state of the game but not return anything except maybe some print statements about the change of states, final_index
        # get pit index corresponding to action taken
        pit_index = self.action2pit(action=action)

        # pick seeds from current pit
        seeds = self.board[pit_index]

        # set seeds from current pit to zero
        self.board[pit_index] = 0

        # iterate over number of seeds picked
        while seeds > 0 :
            global next_index
            print(type(action))
            action += 1
            next_index = self.board_format[action]

            # drop seed at the point found
            self.board[next_index] += 1

            # update number of seeds in hand
            seeds -= 1

            # update current index
            pit_index = next_index

        return next_index

    def capture_seeds(self, action, during_game = True):
        pit_index = self.action2pit(action)
        player_idx = self.current_player - 1
        if self.get_seeds(action) == 4 and np.sum(self.board, axis=None) > 8:
            if during_game:
                if pit_index in self.board_format[player_idx]:
                    self.stores[player_idx] += 4
                    self.set_seeds(action, 0)
                else:
                    self.stores[1 - player_idx] += 4
                    self.board[pit_index] = 0
            else:
                if self.board[pit_index] == 4:
                    self.stores[pit_index] += 4
                    self.set_seeds(action, 0)
        elif self.get_seeds(action) == 4 and np.sum(self.board, axis=None) == 8:
            self.stores[player_idx] += 8
            self.board[self.board > 0] = 0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
b= Board()
b.territory_count
b.capture_seeds(2)
b.board

array([[4., 4., 4., 0., 4., 4.],
       [4., 4., 4., 4., 4., 4.]])

### **Player**

In [ ]:
class Player:
    def __init__(self):
        B = Board()
        self.player1 = 1
        self.player2 = 2
        self.stores = B.stores
        self.territories = B.board_indices
        self.env = GameState()

    def territory_count(self):
        return Board().territory_count

    def territory_indices(self, player):
        player_id = player - 1
        if player == 1:
            return self.territories[:6]
        elif player == 2:
            return self.territories[6:]

    def is_round_winner(self, player):
        player_id = player - 1
        opponent_id = 1 - player_id
        if self.stores[player_id] > self.stores[opponent_id]:
            return True
        return False

    def Player1_step(self):
        action = random.choice(self.env.actions)

### **Game state**

In [ ]:
class GameState:
    def __init__(self):
        self.num_of_rounds = 6
        self.total_games_played = 0
        self.games_won = np.array([0, 0])
        self.current_board_state = Board().board
        self.current_store_state = Board().stores
        self.current_territory_count = Board().territory_count
        self.rounds_completed = 0
        self.turns_completed = 0
        self.win_list = []
        self.round_winner = 0
        self.current_player = 0
        self.other_player = 0

    def update_win_list(self, player):
        if player == self.round_winner:
            self.win_list += [player]

        # evironment_state = np.array()

### **RuleEngine**

In [ ]:
# check for valid actions for players
# check for valid movements
# check for empty pits
# check if is in      players territory

class RuleEngine:
    def __init__(self):
        self.round = 1
        self.turn = 1
        self.actions = np.arange(12)

    def is_valid_action(self, action):
        # check if the action is valid for a player
        if action in self.actions:
            return True
        return False





In [ ]:
num_of_rounds = 12

class GameController:
    def __init__(self, num_of_rounds):
        self.n_players = 2
        self.environment = GameState()
        self.board = Board()
        self.player = Player()

    def starting_player(self, how="random"):
        if how == "random":
            starter = random.sample([1, 2], 1)
            return starter[0]
        elif how == "last_winner":
            starter = self.environment.win_list[-1]
            return starter


    def player

    def game(self):
        players = [self.player.player1, self.player.player2]
        self.environment.current_player = self.starting_player("random")
        self.environment.other_player = 1 if self.environment.current_player == 2 else 2



